In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pwd

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_id = pd.read_csv('ID_Harmonised.csv')

# Ding

In [ ]:
ding = pd.read_excel('metadata_publications/Ding_metadata.xlsx', sheet_name='Table2_Metadata')

In [ ]:
df_id

In [ ]:
ding.tail()

In [ ]:
ding.columns

In [ ]:
ding_dedup = ding[ding.Case.isin(df_id.ID_harmonised.tolist())].drop_duplicates(subset='Case', keep='first')


In [ ]:
interesing_cols = ['Case', 'KRAS_mutation', 'KRAS_type', 'TP53_type', 'SMAD4_type',
       'CDKN2A_type', 'Germline_ATM', 'Germline_BRCA2', 'Germline_FANCC', 'Moffitt_Subtype', 'Bailey_Subtype', 'Collisson_Subtype']

In [ ]:
df_id_merged = df_id.merge(ding_dedup[interesing_cols], left_on='ID_harmonised', right_on='Case', how='left')

In [ ]:
df_id_merged.drop('Dataset_Barcode', axis=1, inplace=True)

In [ ]:
df_id_merged.drop_duplicates()

In [ ]:
ding_2 = pd.read_excel('metadata_publications/Ding_metadata.xlsx', sheet_name='Table1_Clinical_data')

In [ ]:
ding_2_dedup = ding_2[ding_2.Patient_ID.isin(df_id.ID_harmonised.tolist())].drop_duplicates(subset='Patient_ID', keep='first')

In [ ]:
ding_2_dedup.head()

In [ ]:
ding_2_dedup.columns

In [ ]:
# Function to extract the T stage and N stage
def extract_tnm(row):
    # Extract T stage from Pathology_notes
    t_stage = row['Pathology_notes'].split(',')[0]  # Get the first part before the comma
    # Extract N stage from Lymph_node_met
    n_stage = row['Lymph_node_met'].split(',')[1].strip()  # Get the part after the comma
    return f"{t_stage} {n_stage}"

# Create the new TNM classification column
ding_2_dedup['TNM_classification'] = ding_2_dedup.apply(extract_tnm, axis=1)

In [ ]:
interesing_cols = ['Patient_ID', 'Time_from_diagnosis_to_surgery_(weeks)', 'Cycles', 'Location', 'Grade', 'Size_(cm)', 'Stage','Diabetes', 
                   'Pancreatitis', 'Previous_cancer_history',  'Other_co-morbidities', 'Smoker', 'Alcohol', 'Pathology_notes', 'Lymph_node_met',
                   'Survival_post_surgery_(weeks)', 'Vital_status', 'TNM_classification']

In [ ]:
ding_2_dedup.head(2)

In [ ]:
df_id_merged = df_id_merged.merge(ding_2_dedup[interesing_cols], left_on='ID_harmonised', right_on='Patient_ID', how='left')

In [ ]:
df_id_merged.head()

In [ ]:
df_id_merged.shape

# Hwang

In [ ]:
hwang = pd.read_excel('metadata_publications/Hwang_metadata.xlsx', sheet_name='Supplementary Table 1', header=1)

In [ ]:
hwang.head(5)

In [ ]:
hwang['Vital_status'] = hwang['Status Last FUP'].map({
    'DWD': 'Deceased - Disease',
    'DWOD': 'Deceased - Other',
    'MET': 'Alive - Metastatic',
    'NED': 'Alive - Disease-free'
})

In [ ]:
hwang[['TNM_classification', 'Grade']] = hwang['Stage/Grade'].str.split('/', expand=True)

In [ ]:
hwang.rename(columns={'PFS (d)': 'Progression-Free Survival', 'OS (d)': 'Overall Survival'}, inplace=True)

In [ ]:
hwang.head()

In [ ]:
df_id_merged[df_id_merged.Dataset.str.contains('Hwa')]

In [ ]:
df_id_merged[df_id_merged.Dataset.str.contains('Hwa')].shape

In [ ]:
hwang_clean = hwang[hwang['ID'].notna()].copy()

In [ ]:
hwang_clean['ID'] = (
    hwang_clean['ID']
    .astype(str)                   # ensure all are strings
    .str.strip()                   # remove leading/trailing spaces
    .str.replace('†', '', regex=False)  # remove dagger symbol
)

In [ ]:
hwang_clean = hwang_clean[hwang_clean['ID'].str.startswith('PDAC_')]

In [ ]:
hwang_clean.columns

In [ ]:
# df_id_merged['Overall Survival'] 

In [ ]:
df_id_merged['Overall Survival'] = df_id_merged['Survival_post_surgery_(weeks)'] * 7

In [ ]:
# interesing_cols = ['ID', 'Grade', 'TNM_classification' 'Vital_status', 'Overall Survival']

In [ ]:
len(set(hwang_clean.ID) & set(df_id_merged.ID_harmonised))

In [ ]:
hwang_map = hwang_clean.set_index('ID')

In [ ]:
interesing_cols = ['Grade','TNM_classification', 'Vital_status', 'Overall Survival']

In [ ]:
for col in interesing_cols:
    if col in df_id_merged.columns:
        df_id_merged[col] = df_id_merged['ID_harmonised'].map(hwang_map[col]).fillna(df_id_merged[col])
    else:
        df_id_merged[col] = df_id_merged['ID_harmonised'].map(hwang_map[col])    

In [ ]:
df_id_merged.head(3)

In [ ]:
df_id_merged.groupby(['Dataset', 'Grade']).size().unstack().T

In [ ]:
df_id_merged.groupby(['Dataset', 'TNM_classification']).size().unstack().T

# Simeone

In [ ]:
simeone = pd.read_excel('metadata_publications/Simeone_metadata.xlsx', sheet_name='Genomic Mutations')

In [ ]:
simeone #.head(3)

In [ ]:
simeone.rename(columns={'KRAS': 'KRAS_mutation', 'TP53': 'TP53_type', 'CDKN2A': 'CDKN2A_type', 'SMAD4': 'SMAD4_type', 'Source': 'Tissue'}, inplace=True)

In [ ]:
simeone.columns

In [ ]:
df_id_merged[df_id_merged.Dataset.str.contains('Simeo')].head(2)

In [ ]:
interesing_cols = ['Procedure', 'KRAS_mutation',
       'TP53_type', 'CDKN2A_type', 'SMAD4_type']

In [ ]:
simeone_map = simeone.set_index('Patient')

In [ ]:
simeone_map = simeone_map.iloc[:27, :]

In [ ]:
simeone_map

In [ ]:
simeone_map['Unique_ID'] = simeone_map.index.map(dict(zip(df_id_merged[df_id_merged.Dataset.str.contains('Simeo')].ID_harmonised, df_id_merged[df_id_merged.Dataset.str.contains('Simeo')].Unique_ID)))

In [ ]:
simeone_map = simeone_map[~simeone_map.Unique_ID.isna()].set_index('Unique_ID')

In [ ]:
for col in interesing_cols:
    if col in df_id_merged.columns:
        df_id_merged[col] = df_id_merged['Unique_ID'].map(simeone_map[col]).fillna(df_id_merged[col])
    else:
        df_id_merged[col] = df_id_merged['Unique_ID'].map(simeone_map[col])    

In [ ]:
df_id_merged[df_id_merged.Dataset.str.contains('Simeo')].head(2)

In [ ]:
df_id_merged.groupby(['Dataset', 'KRAS_mutation']).size().unstack().T

# Steele

In [ ]:
import scanpy as sc

In [ ]:
pwd

In [ ]:
steele_atlas = sc.read_h5ad('../../Steele_Atlas.h5ad')

In [ ]:
steele_atlas.obs.head(2)

In [ ]:
steele_atlas.obs[steele_atlas.obs['GSE.SRA..Study.'].str.contains('GSE155698')].head(2)

In [ ]:
steele = sc.read_h5ad('/lustre/groups/ml01/workspace/shrey.parikh/PDAC/raw_data/Steele/PDAC_TISSUE_1/filtered_feature_bc_matrix_output.h5ad')

In [ ]:
steele.obs

In [ ]:
# Steele atlas is useless again

In [ ]:
# Check the zenodo 

In [ ]:
zenodo = sc.read_h5ad('/lustre/groups/ml01/workspace/shrey.parikh/PDAC/raw_data/Zenodo/Data/pk_all.h5ad')

In [ ]:
zenodo.obs[zenodo.obs.Project == 'GSE155698'].isna().sum()

### PDAC
- All samples are treatment naive
- Average age 60.2 (47-72)
- Std Dev BMI 23.5 (4.24)
### Adj_Norm
- Average age 66.4 (45-81)
- Std Dev BMI 25 (5.54)

In [ ]:
conditions = [
    (df_id_merged['Dataset'] == 'Steele'),
    (df_id_merged['Dataset'] == 'Steele_Adj_Norm')
]
values = [60.2, 66.4]
df_id_merged['Age'] = np.select(conditions, values, default=np.nan)

In [ ]:
conditions = [
    (df_id_merged['Dataset'] == 'Steele'),
    (df_id_merged['Dataset'] == 'Steele_Adj_Norm')
]
values = [23.5, 25]
df_id_merged['BMI'] = np.select(conditions, values, default=np.nan)

In [ ]:
df_id_merged['Treatment'] = np.where(df_id_merged.Dataset == 'Steele', 'Treatment Naive', np.nan)

In [ ]:
df_id_merged.groupby(['Dataset', 'BMI']).size().unstack()

In [ ]:
df_id_merged.groupby(['Dataset', 'Treatment']).size().unstack()

# Lee

 P2-P5 primary samples came from treatment-naïve patients, while P1 was a second FNA from a patient who had been treated with gemcitabine/paclitaxel. 
For metastatic samples, the most recent therapies prior to sample acquisition were 5-fluorouracil/liposmal irinotecan, evofosfamide/ipilimumab and 
capecitabine for VM, LuM, and PM, respectively. 

In [ ]:
df_id_merged[df_id_merged.Dataset == 'Lee']

### Added already by Daniele

# Peng

In [ ]:
%pip install xlrd

In [ ]:
peng = pd.read_excel('metadata_publications/Peng_metadata.xlsx', header=1)

In [ ]:
peng.columns

In [ ]:
peng.head(2)

In [ ]:
df_id_merged[df_id_merged.Dataset == 'Peng'].head(2)

In [ ]:
peng.rename(columns={'pathologic diagnoses': 'Pathology_notes', 'Staging': 'Stage', 'TNM Classification': 'TNM_classification', 'Age(y/o)': 'Age'}, inplace=True)

In [ ]:
peng.columns

In [ ]:
interesing_cols = ['Pathology_notes', 'Age', 'Procedure', 
       'TNM_classification',  'Stage', 'perineural invasion',
       'vascular invasion', 'peripancreatic infiltarion']

In [ ]:
peng[['Random', 'Unique_ID']] = peng.Number.str.split('-', expand=True)

In [ ]:
df_id_merged.ID_harmonised = np.where(df_id_merged.Dataset.str.contains('Peng'), 'Peng' + '_' + df_id_merged.ID_harmonised, df_id_merged.ID_harmonised)

In [ ]:
# df_id_merged.ID_harmonised = df_id_merged.ID_harmonised.str.replace(r'^Peng_Peng_Peng_', 'Peng_', regex=True)

# df_id_merged.ID_harmonised = df_id_merged.ID_harmonised.str.replace(r'^Peng_Peng_', 'Peng_', regex=True)

In [ ]:
df_id_merged[df_id_merged.Dataset.str.contains('Peng')].head(2)

In [ ]:
peng.Unique_ID = 'Peng' + '_' + peng.Unique_ID

In [ ]:
len(set(peng.Unique_ID))

In [ ]:
len(set(peng.Unique_ID) & set(df_id_merged.ID_harmonised))

In [ ]:
peng_map = peng.iloc[:40, :]

In [ ]:
len(set(peng_map.Unique_ID) & set(df_id_merged.ID_harmonised))

In [ ]:
peng_map = peng_map[~peng_map.Unique_ID.isna()].set_index('Unique_ID')

In [ ]:
peng_map.head(2)

In [ ]:
for col in interesing_cols:
    if col in df_id_merged.columns:
        df_id_merged[col] = df_id_merged['ID_harmonised'].map(peng_map[col]).fillna(df_id_merged[col])
    else:
        df_id_merged[col] = df_id_merged['ID_harmonised'].map(peng_map[col])    

In [ ]:
df_id_merged.groupby(['Dataset', 'TNM_classification']).size().unstack().T

# Caronni

In [ ]:
df_id_merged.Dataset.unique()

In [ ]:
# columns_to_load = ['ID', 'age', 'sex', 'Site of lesion', 'Chemotherapy schedule', 'Grading', 'TNM']
caronni = pd.read_excel('metadata_publications/Caronni_metadata.xlsx', 
                         sheet_name='Patients clinical data')


In [ ]:
df_id_merged.Location

In [ ]:
caronni = caronni[~caronni.ID.isna()]

In [ ]:
caronni.rename(columns={'Site of lesion':'Location', 'TNM':'TNM_classification', 'Grading': 'Grade'}, inplace=True)

In [ ]:
caronni.head()

In [ ]:
df_id_merged[df_id_merged.Dataset == 'Caronni']

In [ ]:
len(set(caronni.ID) & set(df_id_merged.ID_harmonised))

In [ ]:
caronni_map = caronni.set_index('ID')

In [ ]:
interesing_cols = ['Location', 'Grade', 'TNM_classification', 'Tissue']

In [ ]:
for col in interesing_cols:
    if col in df_id_merged.columns:
        df_id_merged[col] = df_id_merged['ID_harmonised'].map(caronni_map[col]).fillna(df_id_merged[col])
    else:
        df_id_merged[col] = df_id_merged['ID_harmonised'].map(caronni_map[col])    

In [ ]:
df_id_merged.groupby(['Dataset', 'TNM_classification']).size().unstack()

# Linn

In [ ]:
linn = pd.read_excel('metadata_publications/Lin_metadata.xlsx')

In [ ]:
linn

In [ ]:
df_id_merged.ID_harmonised = np.where(df_id_merged.Dataset.str.contains('Lin'), 'Lin' + '_' + df_id_merged.ID_harmonised, df_id_merged.ID_harmonised)

In [ ]:
df_id_merged[df_id_merged.Dataset.str.contains('Lin')]

In [ ]:
linn['Patient ID'] = 'Lin' + '_' + linn['Patient ID']

In [ ]:
len(linn['Patient ID'])

In [ ]:
len(set(linn['Patient ID']) & set(df_id_merged.ID_harmonised))

In [ ]:
interesing_cols = ['Stage', 'Grade']

In [ ]:
linn_map = linn.set_index('Patient ID')

In [ ]:
for col in interesing_cols:
    if col in df_id_merged.columns:
        df_id_merged[col] = df_id_merged['ID_harmonised'].map(linn_map[col]).fillna(df_id_merged[col])
    else:
        df_id_merged[col] = df_id_merged['ID_harmonised'].map(linn_map[col])    

In [ ]:
df_id_merged.groupby(['Dataset', 'Stage']).size().unstack()

# Schlesinger

In [ ]:
zenodo[zenodo.obs.Project == 'GSM4293555'].obs

In [ ]:
# no metadata

In [ ]:
df_id_merged.groupby('Dataset').apply(lambda x: x.notna().sum())

In [ ]:
df_id_merged['Treatment'].unique()

In [ ]:
df_id_merged.CDKN2A_type = df_id_merged.CDKN2A_type.replace('WT', 'Wild_Type')
df_id_merged.SMAD4_type = df_id_merged.SMAD4_type.replace('WT', 'Wild_Type')

In [ ]:
# Define the dictionary for harmonizing the 'Location' values
location_dict = {
    'head': 'Head',
    'Head': 'Head',
    'head, uncinate_process': 'Head, Uncinated Process',
    'Head, Uncinated process': 'Head, Uncinated Process',
    'Uncinated process': 'Uncinated Process',
    'neck': 'Neck',
    'neck/body margin': 'Neck/Body',
    'body': 'Body',
    'Body': 'Body',
    'body/tail': 'Body/Tail',
    'Tail': 'Tail',
    'Tail (proximal remanant)': 'Tail, Proximal Remanant',
    'Tail (proximal remanant)': 'Tail, Proximal Remanant',
    'Head/Uncinate': 'Head, Uncinated Process'
}

df_id_merged['Location'] = df_id_merged['Location'].map(location_dict)

In [ ]:
df_id_merged['Location'].unique()

In [ ]:
# Define the dictionary for harmonizing the 'Grade' values
grade_dict = {
    3.0: 'G3',
    1.0: 'G1',
    2.0: 'G2',
    4: 'G4',
    'G3': 'G3',
    'G2': 'G2',
    'g3': 'G3',
    'g2': 'G2',
    'g2-3': 'G2/G3',
    'gX': 'GX'
}

# Harmonize the 'Grade' column in df_id_merged using the dictionary
df_id_merged['Harmonized_Grade'] = df_id_merged['Grade'].map(grade_dict)

In [ ]:
tnm_dict = {
    'ypT2 pN1': 'ypT2 pN1',
    'pT2 pN1': 'pT2 pN1',
    'pT2 pN2': 'pT2 pN2',
    'ypT2 pN0': 'ypT2 pN0',
    'ypT2 pN2': 'ypT2 pN2',
    'pT3 pN2': 'pT3 pN2',
    'ypT3 pN2': 'ypT3 pN2',
    'yT2 pN1': 'yT2 pN1',
    'T4N2M0': 'T4 pN2 M0',
    'T1cN1M0': 'T1c pN1 M0',
    'T2N0M0': 'T2 pN0 M0',
    'T3N0M0': 'T3 pN0 M0',
    'T3N1M0': 'T3 pN1 M0',
    'T1cN2M0': 'T1c pN2 M0',
    'T2N1M0': 'T2 pN1 M0',
    'T3N2M0': 'T3 pN2 M0',
    'T1cN0M0': 'T1c pN0 M0',
    'T2; N1; M0': 'T2 pN1 M0',
    'pT1c, pN2': 'pT1c pN2',
    'T2; N2': 'T2 pN2',
    'T2; N2; M0': 'T2 pN2 M0',
    'T2; N1': 'T2 pN1',
    'ypT2N1M0': 'ypT2 pN1 M0',
    'ypT3N0M0': 'ypT3 pN0 M0',
    'ypT1cN0M0': 'ypT1c pN0 M0',
    'ypT2N0M0': 'ypT2 pN0 M0',
    'ypT1aN1M0': 'ypT1a pN1 M0',
    'ypT2N2M0': 'ypT2 pN2 M0',
    'T2N2M0': 'T2 pN2 M0',
    'ypT1aN0M0': 'ypT1a pN0 M0',
    'ypT1cN1M0': 'ypT1c pN1 M0',
    'ypT3N1M0': 'ypT3 pN1 M0',
    'ypT3 pN0': 'ypT3 pN0',
    'rpT2 moderately differentiated pN1': 'rpT2 pN1',
    'pT2 pN0': 'pT2 pN0',
    'pT1a pN0': 'pT1a pN0',
    'ypT1 moderately differentiated positive for perineural invasion negative for lymphovascular pN0': 'ypT1 pN0'
}

# Harmonize the 'TNM_classification' column using the dictionary
df_id_merged['Harmonized_TNM_Classification'] = df_id_merged['TNM_classification'].map(tnm_dict)

In [ ]:
df_id_merged.to_csv('metadata_publications/metadata_all.csv')

# Reload

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [ ]:
import scanpy as sc

In [ ]:
pwd

In [ ]:
df_id_merged = pd.read_csv('metadata_publications/metadata_all.csv', index_col='Unnamed: 0')

In [ ]:
# change 

In [ ]:
df_id_merged.head(2)

In [ ]:
df = df_id_merged.groupby('Dataset').apply(lambda x: x.notna().sum())

In [ ]:
# Read Anndata

In [ ]:
adata = sc.read_h5ad('../Binned_Data/adata_scanvi_neighbours_leiden.h5ad')

## check shared ID_harmonised

In [ ]:
shared_ids = adata.obs.groupby('ID_harmonised')['Dataset'].nunique()
shared_ids_multiple_datasets = shared_ids[shared_ids > 1]
shared_ids_multiple_datasets_info = adata.obs[adata.obs['ID_harmonised'].isin(shared_ids_multiple_datasets.index)]

In [ ]:
shared_ids_multiple_datasets

In [ ]:
shared_ids_multiple_datasets_info.Dataset.unique()

## Resolve by adding prefixes

In [ ]:
adata.obs['ID_harmonised'] = adata.obs['ID_harmonised'].astype(str)

In [ ]:
conditions = [
    (adata.obs['Dataset'].str.contains('Peng')),
    (adata.obs['Dataset'] == 'Lin')
]
values = ['Peng' + '_' + adata.obs['ID_harmonised'], 'Lin' + '_' + adata.obs['ID_harmonised']]
adata.obs['ID_harmonised'] = np.select(conditions, values, default=adata.obs['ID_harmonised'])

In [ ]:
len(set(df_id_merged.ID_harmonised) & set(adata.obs.ID_harmonised))

## recheck

In [ ]:
shared_ids = adata.obs.groupby('ID_harmonised')['Dataset'].nunique()
shared_ids_multiple_datasets = shared_ids[shared_ids > 1]
shared_ids_multiple_datasets_info = adata.obs[adata.obs['ID_harmonised'].isin(shared_ids_multiple_datasets.index)]

In [ ]:
shared_ids_multiple_datasets

In [ ]:
shared_ids_multiple_datasets

# Check if you can add this biological metadata to obsm

In [ ]:
adata_obs_subset = adata.obs[['Barcode','Dataset','ID_batch_covariate', 'ID_harmonised', 'Unique_ID', 'Level_1_refined','Tissue', 'Age', 'Sex', 'Diabetes', 'Treatment']].copy()

## Add Steele Treatment

In [ ]:
adata_obs_subset.Treatment = np.where(adata_obs_subset.Dataset == 'Steele', 'Treatment Naive', adata_obs_subset.Treatment) 

In [ ]:
df_id_merged.drop('Treatment', axis=1, inplace=True)

In [ ]:
## can drop Case, Patient_ID, Diabetes

In [ ]:
df_id_merged.drop(['Case', 'Patient_ID', 'Diabetes'], axis=1, inplace=True)

## Add Steele and Peng_Normal Age 

In [ ]:
adata_obs_subset.Age = np.where(adata_obs_subset.Dataset == 'Steele', '60s', adata_obs_subset.Age) 

In [ ]:
adata_obs_subset.Age = np.where(adata_obs_subset.Dataset == 'Steele_Adj_Norm', '60s', adata_obs_subset.Age) 

In [ ]:
adata_obs_subset.Age = adata_obs_subset.ID_harmonised.map(dict(zip(df_id_merged[df_id_merged.Dataset == 'Peng'].ID_harmonised, 
         df_id_merged.loc[df_id_merged.Dataset == 'Peng', 'Age']
         .apply(lambda x: f"{int((x // 10) * 10)}s")
         .tolist()))).fillna(adata_obs_subset.Age)

In [ ]:
adata_obs_subset.Age = adata_obs_subset.ID_harmonised.map(dict(zip(df_id_merged[df_id_merged.Dataset == 'Peng_Normal'].ID_harmonised, 
         df_id_merged.loc[df_id_merged.Dataset == 'Peng_Normal', 'Age']
         .apply(lambda x: f"{int((x // 10) * 10)}s")
         .tolist()))).fillna(adata_obs_subset.Age)

In [ ]:
adata_obs_subset[adata_obs_subset.Dataset.str.contains('Peng')].groupby(['Dataset', 'Age']).size().unstack()

In [ ]:
# Drop Age from df_id_merged
df_id_merged.drop('Age', axis=1, inplace=True)

In [ ]:
# Drop Tissue
df_id_merged.drop('Tissue', axis=1, inplace=True)

In [ ]:
# Drop Dataset, Unique_ID, 
df_id_merged.drop(['Dataset', 'Unique_ID'], axis=1, inplace=True)

In [ ]:
biological_metadata = adata_obs_subset.merge(df_id_merged, on='ID_harmonised', how='inner').copy()

In [ ]:
import re

[i for i in biological_metadata.columns if re.search(r'_x|_y', i)]

In [ ]:
biological_metadata.index = adata.obs.index 

In [ ]:
biological_metadata.drop('Barcode', axis=1, inplace=True)

In [ ]:
biological_metadata.

In [ ]:
df = biological_metadata.groupby('Dataset').nunique() #.apply(lambda x: x.notna().nunique())

In [ ]:
counts = {}
for col in df:
    counts[col] = np.count_nonzero(df[col])  
sorted_counts = sorted(counts.items(), key=lambda item: item[1], reverse=True)
for col, count in sorted_counts:
    print(f'{col}: {count}/{len(df[col].index)}')

In [ ]:
biological_metadata.groupby(['Dataset', 'Treatment']).size().unstack().T

# Resolved Ding Treatment

In [ ]:
biological_metadata[biological_metadata.Dataset.str.contains('Ding')].groupby(['ID_harmonised', 'Treatment']).size().unstack() #.fillna(0).apply(pd.to_numeric, errors='coerce')

In [ ]:
ding_ids = biological_metadata[(biological_metadata.Treatment == 'Unknown') & (biological_metadata.Dataset.str.contains('Ding'))].ID_harmonised.unique().tolist() #.groupby('ID_harmonised').size()

In [ ]:
len(ding_ids)

In [ ]:
ding = pd.read_excel('../Downstream/metadata_publications/Ding_metadata.xlsx', sheet_name='Table1_Clinical_data')

In [ ]:
overlapping_ding_id = list(set(ding.Patient_ID) & set(ding_ids))

In [ ]:
ding[ding.Patient_ID.isin(overlapping_ding_id)].Patient_ID

In [ ]:
dict(zip(ding[ding.Patient_ID.isin(overlapping_ding_id)].Patient_ID, ding[ding.Patient_ID.isin(overlapping_ding_id)].Treatment))

In [ ]:
biological_metadata.Treatment.unique().tolist()

In [ ]:
ding_treament_dict = {'HT264P1': 'Treatment Naive',
 'HT270P1': 'Treatment Naive',
 'HT284P1': 'Treatment Naive',
 'HT288P1': 'Treatment Naive',
 'HT224P1': 'FOLIRINOX + Gem/Abraxane',
 'HT231P1': 'FOLFIRINOX + SBRT',
 'HT232P1': 'mFOLFIRINOX + FOLFIRI',
 'HT259P1': 'FOLFIRINOX',
 'HT306P1': 'FOLFIRINOX',
 'HT242P1': 'Unknown',
 'HT412P1': 'FOLFIRONO + Gemcitabine'}

In [ ]:
biological_metadata.Treatment = biological_metadata.ID_harmonised.map(ding_treament_dict).fillna(biological_metadata.Treatment)

In [ ]:
biological_metadata[biological_metadata.Dataset.str.contains('Ding')].groupby(['ID_harmonised', 'Treatment']).size().unstack().fillna(0).apply(pd.to_numeric, errors='coerce').tail(10)

In [ ]:
biological_metadata[(biological_metadata.Dataset.str.contains('Ding')) & (biological_metadata.Treatment == 'Unknown')].ID_harmonised.unique()

In [ ]:
biological_metadata.Treatment = biological_metadata.Treatment.replace('Untreated', 'Treatment Naive')

In [ ]:
biological_metadata.groupby(['Dataset', 'Treatment']).size().unstack().T

# Tissue

In [ ]:
biological_metadata.groupby(['Dataset', 'Tissue']).size().unstack().T

In [ ]:
biological_metadata.Tissue = np.where(biological_metadata.Dataset.str.contains('Ding'), 'Pancreas', biological_metadata.Tissue)

In [ ]:
biological_metadata.Tissue = np.where(biological_metadata.Dataset.str.contains('Peng'), 'Pancreas', biological_metadata.Tissue)

In [ ]:
biological_metadata.Tissue = np.where(biological_metadata.Dataset.str.contains('Schlesinger'), 'Pancreas', biological_metadata.Tissue)

In [ ]:
biological_metadata.Tissue = np.where(biological_metadata.Dataset.str.contains('Steele'), 'Pancreas', biological_metadata.Tissue)

In [ ]:
biological_metadata.groupby(['Dataset', 'Tissue']).size().unstack().T

# Age

In [ ]:
biological_metadata.groupby(['Dataset', 'Age']).size().unstack().T

In [ ]:
ding_age_ids = list(biological_metadata[(biological_metadata.Dataset.str.contains('Ding')) & (biological_metadata.Age == 'Unknown')].ID_harmonised.unique())

In [ ]:
ding.head()

In [ ]:
ding_dict_age = dict(zip(ding.Patient_ID, ding.Age.apply(lambda x: f"{int((x // 10) * 10)}s").tolist()))

In [ ]:
biological_metadata.Age = biological_metadata.ID_harmonised.map(ding_dict_age).fillna(biological_metadata.Age)

In [ ]:
biological_metadata.groupby(['Dataset', 'Age']).size().unstack().T

# Sex

In [ ]:
ding_dict_sex = dict(zip(ding.Patient_ID, ding.Sex.apply(lambda x: x.capitalize())))

In [ ]:
ding_dict_sex

In [ ]:
biological_metadata.Sex = biological_metadata.ID_harmonised.map(ding_dict_sex).fillna(biological_metadata.Sex)

In [ ]:
biological_metadata.groupby(['Dataset', 'Sex']).size().unstack().T

In [ ]:
peng_sex_dict = {'Peng_N1':'Female',
'Peng_N2':'Male',
'Peng_N3':'Male',
'Peng_N4':'Male',
'Peng_N5':'Female',
'Peng_N6':'Female',
'Peng_N7':'Female',
'Peng_N8':'Male',
'Peng_N9':'Male',
'Peng_N10':'Female',
'Peng_N11':'Female'}

In [ ]:
biological_metadata.Sex = biological_metadata.ID_harmonised.map(peng_sex_dict).fillna(biological_metadata.Sex)

In [ ]:
biological_metadata.groupby(['Dataset', 'Sex']).size().unstack().T

# TNM Classification

In [ ]:
biological_metadata.Harmonized_TNM_Classification.unique()

In [ ]:
biological_metadata.Harmonized_TNM_Classification = biological_metadata.Harmonized_TNM_Classification.replace('NaN', 'Unknown')

In [ ]:
biological_metadata.groupby(['Dataset', 'Harmonized_TNM_Classification']).size().unstack().T

# Grade

In [ ]:
biological_metadata.Grade.unique()

In [ ]:
biological_metadata.groupby(['Dataset', 'Grade']).size().unstack().T

In [ ]:
grade_mapping = {
    '1.0': 'G1',
    '2.0': 'G2',
    '3.0': 'G3',
    '1': 'G1',
    '2': 'G2',
    '3': 'G3',
    '4': 'G3',  # Assuming grade 4 maps to G3
    'G1': 'G1',
    'G2': 'G2',
    'G3': 'G3',
    'g1': 'G1',
    'g2': 'G2',
    'g3': 'G3',
    'g2-3': 'G2-G3',  # Assuming g2-3 means G2 or G3
    'gX': 'GX',  # Handle unknown grades
    np.nan: 'Unknown'  # Handle missing values
}
biological_metadata['Harmonized_Grade'] = biological_metadata['Grade'].map(grade_mapping)
biological_metadata.groupby(['Dataset', 'Harmonized_Grade']).size().unstack().T

# Stage

In [ ]:
biological_metadata.Stage.unique()

In [ ]:
biological_metadata.groupby(['Dataset', 'Stage']).size().unstack().T.fillna(0).apply(pd.to_numeric, errors='coerce')

In [ ]:
biological_metadata.Stage = biological_metadata.Stage.replace(np.nan, 'Unknown')

In [ ]:
stage_mapping = {
    'Ia': 'IA',
    'IB': 'IB',
    'IIA': 'IIA',
    'IIB': 'IIB',
    'IIb': 'IIB',
    'III': 'III',
    np.nan: 'Unknown',  # Handle missing values
}
biological_metadata['Harmonized_Stage'] = biological_metadata['Stage'].map(stage_mapping).fillna('Unknown')
biological_metadata.groupby(['Dataset', 'Harmonized_Stage']).size().unstack().T

In [ ]:
simeone_stage = {'P01': 'IV',
'P02': 'IV',
'P03': 'IV',
'P04': 'IB',
'P05': 'III',
'P06': 'III',
'P07': 'IB',
'P08': 'IIB',
'P09': 'IV',
'P10': 'IIB',
'P11': 'IV',
'P12': 'IV',
'P13': 'III',
'P14': 'IB',
'P15': 'III',
'P16': 'IV',
'P17': 'IV',
'P18': 'IV',
'P19': 'IB',
'P20': 'IV',
'P21': 'IV',
'P22': 'III',
'P23': 'IB',
'P24': 'IV',
'P25': 'IV',
'P26': 'IV',
'P26': 'IV',
'P27': 'IV'}

biological_metadata['Harmonized_Stage'] = biological_metadata['ID_harmonised'].map(simeone_stage).fillna(biological_metadata.Harmonized_Stage)
biological_metadata.groupby(['Dataset', 'Harmonized_Stage']).size().unstack().T

In [ ]:
df = biological_metadata.groupby('Dataset').nunique() #.apply(lambda x: x.notna().nunique())
counts = {}
for col in df:
    counts[col] = np.count_nonzero(df[col])  
sorted_counts = sorted(counts.items(), key=lambda item: item[1], reverse=True)
for col, count in sorted_counts:
    print(f'{col}: {count}/{len(df[col].index)}')

# Location

In [ ]:
biological_metadata.Location.unique()

In [ ]:
biological_metadata.groupby(['Dataset', 'Location']).size().unstack().T

In [ ]:
peng_location_mapping = {
    'Peng_T1': 'Body',
    'Peng_T2': 'Head',
    'Peng_T3': 'Uncinated Process',
    'Peng_T4': 'Body',
    'Peng_T5': 'Uncinated Process',
    'Peng_T6': 'Tail',
    'Peng_T7': 'Body',
    'Peng_T8': 'Uncinated Process',
    'Peng_T9': 'Head',
    'Peng_T10': 'Uncinated Process',
    'Peng_T11': 'Body/Tail',
    'Peng_T12': 'Uncinated Process',
    'Peng_T13': 'Head',
    'Peng_T14': 'Head',
    'Peng_T15': 'Head',
    'Peng_T16': 'Body',
    'Peng_T17': 'Body/Tail',
    'Peng_T18': 'Body',
    'Peng_T19': 'Head',
    'Peng_T20': 'Head',
    'Peng_T21': 'Head',
    'Peng_T22': 'Body',
    'Peng_T23': 'Head',
    'Peng_T24': 'Head',
    'Peng_N1': 'Tail',
    'Peng_N2': 'Descending Duodenum',
    'Peng_N3': 'Descending Duodenum',
    'Peng_N4': 'Body/Tail',
    'Peng_N5': 'Body/Tail',
    'Peng_N6': 'Body',
    'Peng_N7': 'Tail',
    'Peng_N8': 'Body/Tail',
    'Peng_N9': 'Tail',
    'Peng_N10': 'Common Bile Duct',
    'Peng_N11': 'Body'
}
biological_metadata['Location'] = biological_metadata['ID_harmonised'].map(peng_location_mapping).fillna(biological_metadata.Location)
biological_metadata.groupby(['Dataset', 'Location']).size().unstack().T

In [ ]:
biological_metadata.groupby(['Dataset', 'BMI']).size().unstack()

In [ ]:
import numpy as np

# Initialize a dictionary to store counts and the datasets for each column
counts = {}
datasets_for_columns = {}

# Loop through columns in the biological_metadata DataFrame
for col in biological_metadata.columns:
    # Exclude 'Unknown' values and get the datasets where values are not 'Unknown'
    non_unknown_data = biological_metadata[biological_metadata[col] != 'Unknown']
    
    # Group by 'Dataset' and count unique values excluding 'Unknown'
    unique_values_per_dataset = non_unknown_data.groupby('Dataset')[col].nunique()

    # Store the number of non-'Unknown' values in each dataset
    counts[col] = np.count_nonzero(unique_values_per_dataset)
    
    # Get the datasets where the value is not 'Unknown'
    datasets_for_columns[col] = unique_values_per_dataset.index.tolist()

# Sort counts in descending order
sorted_counts = sorted(counts.items(), key=lambda item: item[1], reverse=True)

# Print the results
for col, count in sorted_counts:
    print(f'{col}: {count}/{len(biological_metadata["Dataset"].unique())} datasets')
    print(f'Datasets: {", ".join(datasets_for_columns[col])}')

In [ ]:
adata.obsm['biological_metadata'] = biological_metadata.copy()

In [ ]:
adata.obsm['biological_metadata'] = adata.obsm['biological_metadata'].astype(str)

In [ ]:
adata.write('../Binned_Data/adata_scanvi_neighbours_leiden_biological_metadata.h5ad')

In [ ]:
adata.obs.Dataset.unique().tolist()

In [ ]:
biological_metadata.columns

In [ ]:
from collections import Counter

In [ ]:
biological_metadata_only_na = biological_metadata.replace('Unknown', np.nan)

In [ ]:
df = biological_metadata_only_na.groupby('Dataset').nunique() #.apply(lambda x: x.notna().nunique())
counts = {}
for col in df:
    counts[col] = df[df[col] > 0].index.tolist() #np.count_nonzero(df[col])  

In [ ]:
# ANSI escape code for green text
GREEN = '\033[1;32m'
RESET = '\033[0m'

sorted_counts = sorted(counts.items(), key=lambda item: len(item), reverse=False)

for col, count in sorted_counts:
    print(f'{GREEN}{col}{RESET}: {len(count)}/{len(df[col].index)}')
    print(f'Datasets present: {count}')


In [ ]:
biological_metadata

In [ ]:
ding.groupby('Patient_ID')['Treatment'].unique()

In [ ]:
biological_metadata[biological_metadata.Dataset.str.contains('Ding')].groupby('Unique_ID')['Treatment'].unique()

In [ ]:
adata